In [1]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import obspy
import os
import h5py as h5
import re

from seismo_performer import load_test_train_data, seismo_performer_with_spec, model_cnn_spec
from h5_generator import H5Generator, train_test_split as h5_tts

In [2]:
def lol(root_dir,arr:[],fnames:[]):
    for file in os.listdir(root_dir):
        filename = os.fsdecode(file)
        if filename.endswith(".mseed"): 
            fnames.append(re.sub("\.mseed", '', filename))
            df = read(root_dir+"/"+filename)
            arr.append(df)
        if os.path.isdir(root_dir+filename):
            lol(root_dir+filename+"/",arr,fnames)
data = []
fnames = []
lol("./data/",data,fnames)
data

[1 Trace(s) in Stream:
XA.S12.00.MHZ | 1969-12-16T00:00:00.178000Z - 1969-12-17T00:00:03.498755Z | 6.6 Hz, 572423 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-01-09T00:00:00.126000Z - 1970-01-10T00:00:03.446755Z | 6.6 Hz, 572423 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-02-07T00:00:00.571000Z - 1970-02-08T00:00:03.891755Z | 6.6 Hz, 572423 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-02-18T00:00:00.149000Z - 1970-02-19T00:00:02.262208Z | 6.6 Hz, 572415 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-03-14T00:00:00.520000Z - 1970-03-15T00:00:03.840755Z | 6.6 Hz, 572423 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-03-30T00:00:09.390180Z - 1970-03-31T00:00:02.899614Z | 6.6 Hz, 572358 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-04-03T00:00:00.350000Z - 1970-04-04T00:00:03.670755Z | 6.6 Hz, 572423 samples,
 1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-05-20T00:00:00.624000Z - 1970-05-21T00:00:02.133434Z | 6.6 Hz, 572411 samples,
 1 Trace(s) in S

In [3]:
data[0]

1 Trace(s) in Stream:
XA.S12.00.MHZ | 1969-12-16T00:00:00.178000Z - 1969-12-17T00:00:03.498755Z | 6.6 Hz, 572423 samples

In [4]:
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset
from scipy import signal
from matplotlib import cm
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset

In [52]:
from scipy.signal import butter,lfilter, freqz

def butter_lowpass(data, cutoff, fs, order=5):
  nyq=0.5*fs #Frecuencia de Nyquist
  normal_cutoff = cutoff/nyq #Frecuencia normalizada 
  b,a = butter(order, normal_cutoff, btype='low', analog=False)
  y=lfilter(b,a,data)
  return y

def pipe(data:[]):

    sta_len = 120
    lta_len = 600
    thr_on = 4
    thr_off = 1.5
    index = 0
    nfnames = fnames.copy()

    catalogue = pd.DataFrame(columns = ['filename', 'time_abs(%Y-%m-%dT%H:%M:%S.%f)', 'time_rel(sec)'])
    ret = []
    for mseed in data:
        print(mseed)
        fname = nfnames[index]
        df = mseed[0].stats.sampling_rate
        st_filt = mseed.copy()
        
        #st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
        tr_filt = st_filt.traces[0].copy()
        
        tr_times = tr_filt.times()
        tr_data =  butter_lowpass(tr_filt.data, 0.6, df,10)
        starttime = tr_filt.stats.starttime.datetime
        
        cft = classic_sta_lta(tr_data, int(sta_len * df), int(lta_len * df))

        on_off = np.array(trigger_onset(cft, thr_on, thr_off))
    
        f, t, sxx = signal.spectrogram(tr_data, tr_filt.stats.sampling_rate)

        # Plot the time series and spectrogram
        fig = plt.figure(figsize=(10, 10))
        ax = plt.subplot(2, 1, 1)
# Plot trace
        ax.plot(tr_times,tr_data)

# Mark detection
        detection_times = []
        names =[]
        for i in np.arange(0,len(on_off)):
            triggers = on_off[i]
            ax.axvline(x = tr_times[triggers[0]], color='red', label='Trig. On')
            ax.axvline(x = tr_times[triggers[1]], color='purple', label='Trig. Off')
       
            triggers = on_off[i]
            on_time = starttime + timedelta(seconds = tr_times[triggers[0]])
            on_time_str = datetime.strftime(on_time,'%Y-%m-%dT%H:%M:%S.%f')
            detection_times.append(on_time_str)
            names.append(fname)
        print(len(names),len(detection_times),tr_times[triggers[0]].shape)
        detect_df = pd.DataFrame(data = {'filename':names, 'time_abs(%Y-%m-%dT%H:%M:%S.%f)':detection_times, 'time_rel(sec)':tr_times[triggers[0]]})
        catalogue =pd.concat([catalogue, detect_df], ignore_index=True)


# Make the plot pretty
        ax.set_xlim([min(tr_times),max(tr_times)])
        ax.set_ylabel('Velocity (m/s)')
        ax.set_xlabel('Time (s)')

        ax2 = plt.subplot(2, 1, 2)
        vals = ax2.pcolormesh(t, f, sxx, cmap=cm.jet, vmax=5e-17)
        ax2.set_xlim([min(tr_times),max(tr_times)])
        ax2.set_xlabel(f'Time (Day Hour:Minute)', fontweight='bold')
        ax2.set_ylabel('Frequency (Hz)', fontweight='bold')
        #ax2.axvline(x=arrival, c='red')
        cbar = plt.colorbar(vals, orientation='horizontal')
        cbar.set_label('Power ((m/s)^2/sqrt(Hz))', fontweight='bold')

        plt.savefig("./plots/"+str(index)+'.png')
        plt.close()
        index = index + 1

    catalogue.to_csv("catalogue.csv", sep=',', encoding='utf-8',index= False)


In [53]:
pipe(data)


1 Trace(s) in Stream:
XA.S12.00.MHZ | 1969-12-16T00:00:00.178000Z - 1969-12-17T00:00:03.498755Z | 6.6 Hz, 572423 samples
24 24 ()


C:\Users\Dell\AppData\Local\Temp\ipykernel_6212\3871889193.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  catalogue =pd.concat([catalogue, detect_df], ignore_index=True)


1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-01-09T00:00:00.126000Z - 1970-01-10T00:00:03.446755Z | 6.6 Hz, 572423 samples
21 21 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-02-07T00:00:00.571000Z - 1970-02-08T00:00:03.891755Z | 6.6 Hz, 572423 samples
17 17 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-02-18T00:00:00.149000Z - 1970-02-19T00:00:02.262208Z | 6.6 Hz, 572415 samples
14 14 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-03-14T00:00:00.520000Z - 1970-03-15T00:00:03.840755Z | 6.6 Hz, 572423 samples
3 3 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-03-30T00:00:09.390180Z - 1970-03-31T00:00:02.899614Z | 6.6 Hz, 572358 samples
8 8 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-04-03T00:00:00.350000Z - 1970-04-04T00:00:03.670755Z | 6.6 Hz, 572423 samples
11 11 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-05-20T00:00:00.624000Z - 1970-05-21T00:00:02.133434Z | 6.6 Hz, 572411 samples
18 18 ()
1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-05-23T00:00:00.147000Z - 1970-05-24T00:00:02.260208